<a href="https://colab.research.google.com/github/HirunaD/LangChain/blob/main/05_Conversational_RAG_Application.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Conversational RAG Application with LangChain and OpenAI **LLM**

In [1]:
# Install the necessary packages
!pip install langchain -qU
!pip install langchain-openai -qU
!pip install langchain-chroma -qU
!pip install langchain_community -qU

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.0/69.0 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 63.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 73.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.2/196.2 kB 11.9 MB/s eta 0:

In [2]:
import os
from google.colab import userdata

**Initialize OpenAI LLM**

In [3]:
from langchain_openai import ChatOpenAI

# Set OpenAI API key
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

# Initialize the ChatOpenAI model
llm = ChatOpenAI(
    model="gpt-4.1-nano",
    temperature=0
)

In [4]:
from langchain_openai import OpenAIEmbeddings
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

**Load PDF Document**

In [9]:
!pip install pdfplumber -qU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 61.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 74.3 MB/s eta 0:00:00


In [12]:
from langchain_community.document_loaders import PyPDFLoader

# Load the PDF document
file_path = "/content/codeprolk.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()

In [13]:
len(docs)

4

**Split Documents into Chunks**

In [14]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Initialize the text splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=50)

# Split the documents into chunks
splits = text_splitter.split_documents(docs)

In [15]:
len(splits)

20

**Create Vector Store and Retriever**

In [16]:
from langchain_chroma import Chroma

# Create a vector store from the document chunks
vectorstore = Chroma.from_documents(documents=splits, embedding=embedding_model)

In [17]:
# Create a retriever from the vector store
retriever = vectorstore.as_retriever()

**Define Prompt Template**

In [18]:
from langchain_core.prompts import ChatPromptTemplate

# Define the system prompt
system_prompt = (
    "You are an intelligent chatbot. Use the following context to answer the question. If you don't know the answer, just say that you don't know."
    "\n\n"
    "{context}"
)

# Create the prompt template
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [19]:
prompt

ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template="You are an intelligent chatbot. Use the following context to answer the question. If you don't know the answer, just say that you don't know.\n\n{context}"), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])

**Create Retrieval-Augmented Generation (RAG) Chain**

In [20]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

# Create the question-answering chain
qa_chain = create_stuff_documents_chain(llm, prompt)

# Create the RAG chain
rag_chain = create_retrieval_chain(retriever, qa_chain)

**Invoke RAG Chain with Example Questions**

In [21]:
response = rag_chain.invoke({"input": "who is codeprolk"})
response["answer"]

'CodePRO LK is a dynamic educational platform based in Sri Lanka that offers a variety of technology-related courses in Sinhala. It aims to empower Sri Lankans with skills in programming, data science, and machine learning. Founded by Dinesh Piyasamara during the COVID-19 pandemic, the platform focuses on providing accessible, high-quality tech education and continuously expanding its offerings through partnerships, community engagement, and events.'

In [22]:
response = rag_chain.invoke({"input": "what is rag architecture"})
response["answer"]

"I don't know."

In [23]:
response = rag_chain.invoke({"input": "what are the courses codeprolk offer"})
response["answer"]

'CodePRO LK offers a variety of free courses in Sinhala, catering to different proficiency levels from beginners to intermediates. One key course mentioned is "Python GUI – Tkinter," which covers the essentials of creating graphical user interfaces. The platform plans to expand its offerings to include more advanced topics such as artificial intelligence, cybersecurity, and data analytics in the future.'

In [24]:
response = rag_chain.invoke({"input": "can you list down"})
response["answer"]

'Sure! Please specify what you would like me to list down.'

**Add Chat History**

In [25]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

# Define the contextualize system prompt
contextualize_system_prompt = (
    "using chat history and the latest user question, just reformulate question if needed and otherwise return it as is"
)

# Create the contextualize prompt template
contextualize_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

# Create the history-aware retriever
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_prompt
)

**Create History-Aware RAG Chain**

In [26]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

system_prompt = (
    "You are an intelligent chatbot. Use the following context to answer the question. If you don't know the answer, just say that you don't know."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

prompt

ChatPromptTemplate(input_variables=['chat_history', 'context', 'input'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.

In [27]:
# Create the question-answering chain
qa_chain = create_stuff_documents_chain(llm, prompt)

# Create the history aware RAG chain
rag_chain = create_retrieval_chain(history_aware_retriever, qa_chain)

**Manage Chat Session History**

In [28]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# Initialize the store for session histories
store = {}

# Function to get the session history for a given session ID
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

# Create the conversational RAG chain with session history
conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

**Invoke Conversational RAG Chain with Example Questions**

In [29]:
response = conversational_rag_chain.invoke(
    {"input": "who is codeprolk"},
    config={"configurable": {"session_id": "101"}},
)
response["answer"]

'CodePRO LK is a dynamic educational platform based in Sri Lanka that offers a variety of technology-related courses in Sinhala. It aims to empower Sri Lankans with skills in programming, data science, and machine learning. Founded by Dinesh Piyasamara during the COVID-19 pandemic, the platform focuses on providing accessible, high-quality tech education and fostering a supportive community through events like webinars, hackathons, and tech talks.'

In [30]:
store

{'101': InMemoryChatMessageHistory(messages=[HumanMessage(content='who is codeprolk', additional_kwargs={}, response_metadata={}), AIMessage(content='CodePRO LK is a dynamic educational platform based in Sri Lanka that offers a variety of technology-related courses in Sinhala. It aims to empower Sri Lankans with skills in programming, data science, and machine learning. Founded by Dinesh Piyasamara during the COVID-19 pandemic, the platform focuses on providing accessible, high-quality tech education and fostering a supportive community through events like webinars, hackathons, and tech talks.', additional_kwargs={}, response_metadata={})])}

In [31]:
response = conversational_rag_chain.invoke(
    {"input": "what is rag architecture"},
    config={"configurable": {"session_id": "101"}},
)
response["answer"]

"I don't know."

In [32]:
response = conversational_rag_chain.invoke(
    {"input": "what are the courses codeprolk offer"},
    config={"configurable": {"session_id": "101"}},
)
response["answer"]

'CodePRO LK offers a variety of free courses in Sinhala that cater to different proficiency levels, from beginners to intermediates. Some of the key courses include:\n\n- Python GUI – Tkinter: This course covers the essentials of creating graphical user interfaces using Python and Tkinter.\n\nThe platform is also planning to expand its offerings to include more advanced topics such as artificial intelligence, cybersecurity, and data analytics in the future.'

In [33]:
response = conversational_rag_chain.invoke(
    {"input": "can you list down"},
    config={"configurable": {"session_id": "101"}},
)
response["answer"]

'Based on the provided information, only the Python GUI – Tkinter course is explicitly mentioned. However, it is likely that CodePRO LK offers other courses related to programming, data science, and machine learning, given their focus areas. The platform also plans to expand into advanced topics like artificial intelligence, cybersecurity, and data analytics in the future.'